# Data Preprocessing Notebook
## Ford GoBike Dataset Cleaning and Preparation

This notebook preprocesses, cleans, and prepares the Ford GoBike dataset for analysis.

**Objectives:**
1. Load and explore the raw dataset
2. Handle missing values
3. Fix data types
4. Create derived features (age, duration in minutes/hours) -i.e. feature engineering-
5. Handle outliers
6. Remove duplicates
7. Export cleaned dataset to `data/processed/`


## 1. Import Libraries


In [1]:
import pandas as pd
import warnings
from pathlib import Path
import sys
import os
from datetime import datetime

warnings.filterwarnings('ignore')
pd.set_option('display.max_rows', 100)

PROJECT_ROOT = Path.cwd().parent
sys.path.append(str(PROJECT_ROOT))

from src.utils import processing_utils as pu

## 2. Load Raw Data


In [2]:
raw_data_path = Path('../data/raw/fordgobike_raw.csv')

df = pu.load_data(raw_data_path)

data_info = pu.get_data_info(df)

print("######## QUICK OVERVIEW OF THE DATASET ########")
print(f"\nDataset shape: No. of Entries: {data_info['shape'][0]}, No. of Columns: {data_info['shape'][1]}")
print(f"\nColumns: {data_info['columns']}")
print(f"\nFirst few rows:")

display(data_info['head'])

######## QUICK OVERVIEW OF THE DATASET ########

Dataset shape: No. of Entries: 183416, No. of Columns: 16

Columns: ['duration_sec', 'start_time', 'end_time', 'start_station_id', 'start_station_name', 'start_station_latitude', 'start_station_longitude', 'end_station_id', 'end_station_name', 'end_station_latitude', 'end_station_longitude', 'bike_id', 'user_type', 'member_birth_year', 'member_gender', 'bike_share_for_all_trip']

First few rows:


,duration_sec,start_time,end_time,start_station_id,start_station_name,start_station_latitude,start_station_longitude,end_station_id,end_station_name,end_station_latitude,end_station_longitude,bike_id,user_type,member_birth_year,member_gender,bike_share_for_all_trip
0,52185,32:10.1,01:56.0,21.0,Montgomery St BART Station (Market St at 2nd St),37.789625,-122.400811,13.0,Commercial St at Montgomery St,37.794231,-122.402923,4902,Customer,1984.0,Male,No
1,42521,53:21.8,42:03.1,23.0,The Embarcadero at Steuart St,37.791464,-122.391034,81.0,Berry St at 4th St,37.775880,-122.393170,2535,Customer,NaN,NaN,No
2,61854,13:13.2,24:08.1,86.0,Market St at Dolores St,37.769305,-122.426826,3.0,Powell St BART Station (Market St at 4th St),37.786375,-122.404904,5905,Customer,1972.0,Male,No
3,36490,54:26.0,02:36.8,375.0,Grove St at Masonic Ave,37.774836,-122.446546,70.0,Central Ave at Fell St,37.773311,-122.444293,6638,Subscriber,1989.0,Other,No
4,1585,54:18.5,20:44.1,7.0,Frank H Ogawa Plaza,37.804562,-122.271738,222.0,10th Ave at E 15th St,37.792714,-122.248780,4898,Subscriber,1974.0,Male,Yes


## 3. Initial Data Exploration


In [3]:
display(pu.get_data_types(df))

,dtype
duration_sec,int64
start_time,object
end_time,object
start_station_id,float64
start_station_name,object
start_station_latitude,float64
start_station_longitude,float64
end_station_id,float64
end_station_name,object
end_station_latitude,float64


In [4]:
missing_values = df.isnull().sum()

print("Missing Values:\n")

print(missing_values[missing_values > 0])
print(f"\nTotal missing values: {df.isnull().sum().sum()}")
print(f"Percentage of missing values: {(df.isnull().sum().sum() / len(df) * 100):.2f}%")

Missing Values:

start_station_id       197
start_station_name     197
end_station_id         197
end_station_name       197
member_birth_year     8265
member_gender         8265
dtype: int64

Total missing values: 17318
Percentage of missing values: 9.44%


In [5]:
print(f"Total duplicate rows: {df.duplicated().sum()}")
print(f"Percentage of duplicates: {(df.duplicated().sum() / len(df) * 100):.10f}%")

Total duplicate rows: 4
Percentage of duplicates: 0.0021808348%


In [6]:
print("Unique values in categorical columns:")

print(f"\nuser_type: {df['user_type'].unique()}")
print(f"\nmember_gender: {df['member_gender'].unique()}")
print(f"\nbike_share_for_all_trip: {df['bike_share_for_all_trip'].unique()}")

Unique values in categorical columns:

user_type: ['Customer' 'Subscriber']

member_gender: ['Male' nan 'Other' 'Female']

bike_share_for_all_trip: ['No' 'Yes']


In [7]:
print("Basic Statistics:")

df.describe()

Basic Statistics:


,duration_sec,start_station_id,start_station_latitude,start_station_longitude,end_station_id,end_station_latitude,end_station_longitude,bike_id,member_birth_year
count,183416.000000,183219.000000,183416.000000,183416.000000,183219.000000,183416.000000,183416.000000,183416.000000,175151.000000
mean,726.067311,138.590365,37.771221,-122.352661,136.249401,37.771425,-122.352247,4472.890260,1984.806504
std,1794.371831,111.778938,0.099586,0.117100,111.514916,0.099494,0.116677,1664.386068,10.116589
min,61.000000,3.000000,37.317298,-122.453705,3.000000,37.317298,-122.453705,11.000000,1878.000000
25%,325.000000,47.000000,37.770083,-122.412408,44.000000,37.770407,-122.411726,3777.000000,1980.000000
50%,514.000000,104.000000,37.780760,-122.398285,100.000000,37.781010,-122.398279,4958.000000,1987.000000
75%,796.000000,239.000000,37.797280,-122.286533,235.000000,37.797320,-122.288045,5502.000000,1992.000000
max,85444.000000,398.000000,37.880222,-121.874119,398.000000,37.880222,-121.874119,6645.000000,2001.000000


## 4. Data Cleaning

### 4.1 Create a copy for cleaning


In [8]:
df_clean = df.copy()

print(f"Original shape: {df.shape}")
print(f"Clean copy shape: {df_clean.shape}")

Original shape: (183416, 16)
Clean copy shape: (183416, 16)


### 4.2 Handle Missing Values

#### 4.2.1 Handle missing member_gender


In [9]:
print(f"Missing values in member_gender: {df_clean['member_gender'].isnull().sum()}")
print(f"\nDistribution before filling:")
print(df_clean['member_gender'].value_counts(dropna=False))

df_clean = pu.fill_missing_categorical(df_clean, ['member_gender'], 'Unknown')
print(f"\nMissing values after filling: {df_clean['member_gender'].isnull().sum()}")

Missing values in member_gender: 8265

Distribution before filling:
member_gender
Male      130655
Female     40844
NaN         8265
Other       3652
Name: count, dtype: int64

Missing values after filling: 0


#### 4.2.2 Handle missing member_birth_year


In [10]:
print(f"\nMissing values in member_birth_year: {df_clean['member_birth_year'].isnull().sum()}")
print(f"\nStatistics for member_birth_year:")
print(df_clean['member_birth_year'].describe())

current_year = datetime.now().year
print(f"\nCurrent year: {current_year}")
print(f"Min birth year: {df_clean['member_birth_year'].min()}")
print(f"Max birth year: {df_clean['member_birth_year'].max()}")

median_birth_year = df_clean['member_birth_year'].median()
print(f"\nMedian birth year: {median_birth_year}")

df_clean = pu.fill_missing_numeric(df_clean, ['member_birth_year'], method='median')
print(f"Missing values after filling: {df_clean['member_birth_year'].isnull().sum()}")


Missing values in member_birth_year: 8265

Statistics for member_birth_year:
count    175151.000000
mean       1984.806504
std          10.116589
min        1878.000000
25%        1980.000000
50%        1987.000000
75%        1992.000000
max        2001.000000
Name: member_birth_year, dtype: float64

Current year: 2026
Min birth year: 1878.0
Max birth year: 2001.0

Median birth year: 1987.0
Missing values after filling: 0


#### 4.2.3 Handle missing station information


In [11]:
print("\nMissing values in station columns:")

station_cols = ['start_station_id', 'start_station_name', 'end_station_id', 'end_station_name']

for col in station_cols:
    print(f"{col}: {df_clean[col].isnull().sum()}")

print(f"\nRows with missing start station info: {df_clean['start_station_id'].isnull().sum()}")
print(f"Rows with missing end station info: {df_clean['end_station_id'].isnull().sum()}")

df_clean = pu.fill_missing_with_mapping(df_clean, {
    'start_station_id': -1,
    'start_station_name': 'Unknown',
    'end_station_id': -1,
    'end_station_name': 'Unknown'
})


Missing values in station columns:
start_station_id: 197
start_station_name: 197
end_station_id: 197
end_station_name: 197

Rows with missing start station info: 197
Rows with missing end station info: 197


In [12]:
print("\nAfter filling:")

for col in station_cols:
    print(f"{col}: {df_clean[col].isnull().sum()}")


After filling:
start_station_id: 0
start_station_name: 0
end_station_id: 0
end_station_name: 0


### 4.3 Fix Data Types

#### 4.3.1 Convert categorical columns to category type


In [13]:
categorical_cols = ['user_type', 'member_gender', 'bike_share_for_all_trip']

print("Before conversion:")
for col in categorical_cols:
    print(f"{col}: {df_clean[col].dtype}")

df_clean = pu.convert_to_category(df_clean, categorical_cols)

print("\nAfter conversion:")
for col in categorical_cols:
    print(f"{col}: {df_clean[col].dtype}")

Before conversion:
user_type: object
member_gender: object
bike_share_for_all_trip: object

After conversion:
user_type: category
member_gender: category
bike_share_for_all_trip: category


#### 4.3.2 Convert station IDs to integer (where not missing)


In [14]:
df_clean = pu.convert_to_int(df_clean, ['start_station_id', 'end_station_id'])

print("\nStation ID data types:")
print(f"start_station_id: {df_clean['start_station_id'].dtype}")
print(f"end_station_id: {df_clean['end_station_id'].dtype}")


Station ID data types:
start_station_id: int64
end_station_id: int64


### 4.4 Create Derived Features (Feature Engineering)

#### 4.4.1 Create age column from member_birth_year


In [15]:
df_clean = pu.calculate_age_from_year(df_clean, 'member_birth_year', 'age')

print("Age statistics:")
print(df_clean['age'].describe())
print(f"\nAge range: {df_clean['age'].min()} to {df_clean['age'].max()}")

Age statistics:
count    183416.000000
mean         41.094654
std           9.896492
min          25.000000
25%          34.000000
50%          39.000000
75%          45.000000
max         148.000000
Name: age, dtype: float64

Age range: 25 to 148


#### 4.4.2 Create duration in minutes and hours


In [16]:
df_clean = pu.convert_duration(df_clean, 'duration_sec', 'duration_minutes', 'duration_hours', decimals=2)

print("\nDuration statistics:")
print(f"\nDuration in seconds:")
print(df_clean['duration_sec'].describe())
print(f"\nDuration in minutes:")
print(df_clean['duration_minutes'].describe())
print(f"\nDuration in hours:")
print(df_clean['duration_hours'].describe())

df_clean = pu.flag_extreme_values(df_clean, 'duration_sec', 'is_extreme_duration', lower_bound=60, upper_bound=86400)
print(f"\nExtreme duration trips flagged: {df_clean['is_extreme_duration'].sum()}")



Duration statistics:

Duration in seconds:
count    183416.000000
mean        726.067311
std        1794.371831
min          61.000000
25%         325.000000
50%         514.000000
75%         796.000000
max       85444.000000
Name: duration_sec, dtype: float64

Duration in minutes:
count    183416.000000
mean         12.101115
std          29.906202
min           1.020000
25%           5.420000
50%           8.570000
75%          13.270000
max        1424.070000
Name: duration_minutes, dtype: float64

Duration in hours:
count    183416.000000
mean          0.201693
std           0.498451
min           0.020000
25%           0.090000
50%           0.140000
75%           0.220000
max          23.730000
Name: duration_hours, dtype: float64

Extreme duration trips flagged: 0


### 4.5 Handle Outliers

#### 4.5.1 Handle age outliers


In [17]:
age_outliers = (df_clean['age'] < 0) | (df_clean['age'] > 120)
print(f"Age outliers (< 0 or > 120): {age_outliers.sum()}")

if age_outliers.sum() > 0:
    print("\nOutlier ages:")
    print(df_clean[age_outliers][['member_birth_year', 'age']].head(10))
    df_clean = pu.cap_outliers(df_clean, 'age', lower_bound=0, upper_bound=120, method='median')
    print(f"\nAfter handling outliers, age range: {df_clean['age'].min()} to {df_clean['age'].max()}")
else:
    print("No age outliers found!")



Age outliers (< 0 or > 120): 71

Outlier ages:
       member_birth_year  age
1285              1900.0  126
10827             1900.0  126
16087             1900.0  126
19375             1900.0  126
21424             1900.0  126
21506             1900.0  126
24042             1900.0  126
26133             1900.0  126
27370             1878.0  148
27675             1900.0  126

After handling outliers, age range: 25 to 116


#### 4.5.2 Handle duration outliers


In [18]:
short_trips = df_clean['duration_sec'] < 60
long_trips = df_clean['duration_sec'] > 86400
print(f"\nVery short trips (< 60 seconds): {short_trips.sum()}")
print(f"Very long trips (> 24 hours): {long_trips.sum()}")
print(f"\nDuration percentiles:")
print(df_clean['duration_sec'].quantile([0.01, 0.05, 0.25, 0.5, 0.75, 0.95, 0.99]))


Very short trips (< 60 seconds): 0
Very long trips (> 24 hours): 0

Duration percentiles:
0.01     104.00
0.05     167.00
0.25     325.00
0.50     514.00
0.75     796.00
0.95    1571.00
0.99    3458.85
Name: duration_sec, dtype: float64


### 4.6 Remove Duplicates


In [19]:
# Remove duplicates
duplicates_before = df_clean.duplicated().sum()
print(f"Duplicate rows before removal: {duplicates_before}")

if duplicates_before > 0:
    df_clean = pu.remove_duplicates(df_clean)
    print(f"Duplicate rows after removal: {df_clean.duplicated().sum()}")
    print(f"Rows removed: {duplicates_before}")
else:
    print("No duplicate rows found!")


Duplicate rows before removal: 4
Duplicate rows after removal: 0
Rows removed: 4


## 5. Final Data Quality Check


In [20]:
print("Final Missing Values Check:")

missing_final = df_clean.isnull().sum()

print(missing_final[missing_final > 0])

if missing_final.sum() == 0:
    print("\n✓ No missing values remaining!")
else:
    print(f"\n⚠ Still have {missing_final.sum()} missing values")


Final Missing Values Check:
Series([], dtype: int64)

✓ No missing values remaining!


In [21]:
print("Final Data Types:")
display(pd.DataFrame({"d_type": df_clean.dtypes}))

Final Data Types:


,d_type
duration_sec,int64
start_time,object
end_time,object
start_station_id,int64
start_station_name,object
start_station_latitude,float64
start_station_longitude,float64
end_station_id,int64
end_station_name,object
end_station_latitude,float64


In [22]:
print("Final Dataset Shape:", df_clean.shape)
print("\nSummary Statistics:")

df_clean.describe()

Final Dataset Shape: (183412, 20)

Summary Statistics:


,duration_sec,start_station_id,start_station_latitude,start_station_longitude,end_station_id,end_station_latitude,end_station_longitude,bike_id,member_birth_year,age,duration_minutes,duration_hours
count,183412.000000,183412.000000,183412.000000,183412.000000,183412.000000,183412.000000,183412.000000,183412.000000,183412.000000,183412.000000,183412.000000,183412.000000
mean,726.078435,138.440495,37.771223,-122.352664,136.101705,37.771427,-122.352250,4472.906375,1984.905284,41.061070,12.101301,0.201696
std,1794.389780,111.812347,0.099581,0.117097,111.545859,0.099490,0.116673,1664.383394,9.896585,9.754741,29.906501,0.498456
min,61.000000,-1.000000,37.317298,-122.453705,-1.000000,37.317298,-122.453705,11.000000,1878.000000,25.000000,1.020000,0.020000
25%,325.000000,47.000000,37.770083,-122.412408,44.000000,37.770407,-122.411726,3777.000000,1981.000000,34.000000,5.420000,0.090000
50%,514.000000,104.000000,37.780760,-122.398285,100.000000,37.781010,-122.398279,4958.000000,1987.000000,39.000000,8.570000,0.140000
75%,796.000000,239.000000,37.797280,-122.286533,235.000000,37.797320,-122.288045,5502.000000,1992.000000,45.000000,13.270000,0.220000
max,85444.000000,398.000000,37.880222,-121.874119,398.000000,37.880222,-121.874119,6645.000000,2001.000000,116.000000,1424.070000,23.730000


In [23]:
print("Sample of cleaned data:")

df_clean.sample(10)

Sample of cleaned data:


,duration_sec,start_time,end_time,start_station_id,start_station_name,start_station_latitude,start_station_longitude,end_station_id,end_station_name,end_station_latitude,end_station_longitude,bike_id,user_type,member_birth_year,member_gender,bike_share_for_all_trip,age,duration_minutes,duration_hours,is_extreme_duration
53264,572,05:27.8,15:00.2,81,Berry St at 4th St,37.775880,-122.393170,58,Market St at 10th St,37.776619,-122.417385,6573,Subscriber,1992.0,Male,No,34,9.53,0.16,False
2339,988,01:44.2,18:12.2,36,Folsom St at 3rd St,37.783830,-122.398870,127,Valencia St at 21st St,37.756708,-122.421025,4711,Subscriber,1987.0,Male,No,39,16.47,0.27,False
102986,350,09:59.2,15:49.8,350,8th St at Brannan St,37.771431,-122.405787,350,8th St at Brannan St,37.771431,-122.405787,4631,Customer,1990.0,Male,No,36,5.83,0.10,False
57486,339,17:31.9,23:11.0,113,Franklin Square,37.764555,-122.410345,223,16th St Mission BART Station 2,37.764765,-122.420091,204,Subscriber,1998.0,Male,No,28,5.65,0.09,False
43439,764,29:04.6,41:49.4,81,Berry St at 4th St,37.775880,-122.393170,5,Powell St BART Station (Market St at 5th St),37.783899,-122.408445,2429,Subscriber,1978.0,Male,No,48,12.73,0.21,False
84769,302,49:59.1,55:01.5,159,24th St at Market St,37.816060,-122.278244,183,Telegraph Ave at 19th St,37.808702,-122.269927,5443,Subscriber,1977.0,Male,No,49,5.03,0.08,False
72191,1804,25:31.1,55:35.5,23,The Embarcadero at Steuart St,37.791464,-122.391034,31,Raymond Kimbell Playground,37.783813,-122.434559,6119,Subscriber,1995.0,Female,No,31,30.07,0.50,False
130090,920,35:22.3,50:42.4,29,O'Farrell St at Divisadero St,37.782405,-122.439446,350,8th St at Brannan St,37.771431,-122.405787,5047,Subscriber,1997.0,Female,No,29,15.33,0.26,False
68393,1604,45:03.6,11:48.3,22,Howard St at Beale St,37.789756,-122.394643,368,Myrtle St at Polk St,37.785434,-122.419622,5941,Customer,1987.0,Unknown,No,39,26.73,0.45,False
31111,314,36:14.8,41:29.5,100,Bryant St at 15th St,37.767100,-122.410662,114,Rhode Island St at 17th St,37.764478,-122.402570,1920,Subscriber,1986.0,Female,No,40,5.23,0.09,False


## 6. Export Cleaned Dataset


In [24]:
output_path = pu.export_data(df_clean, '../data/processed', 'fordgobike_processed.csv')
file_size_mb = os.path.getsize(output_path) / (1024 * 1024)

print(f"Cleaned dataset exported to: {output_path}")
print(f"Dataset shape: {df_clean.shape}")
print(f"File size: {file_size_mb:.2f} MB")

Cleaned dataset exported to: ..\data\processed\fordgobike_processed.csv
Dataset shape: (183412, 20)
File size: 31.63 MB


## 7. Summary of Preprocessing Steps

### Data Cleaning Summary:
1. ✓ Handled missing values:
   - `member_gender`: Filled with 'Unknown'
   - `member_birth_year`: Filled with median value
   - Station information: Filled with 'Unknown' for names and -1 for IDs

2. ✓ Fixed data types:
   - Converted categorical columns (`user_type`, `member_gender`, `bike_share_for_all_trip`) to category type
   - Converted station IDs to integer

3. ✓ Created derived features:
   - `age`: Calculated from `member_birth_year`
   - `duration_minutes`: Duration in minutes
   - `duration_hours`: Duration in hours
   - `is_extreme_duration`: Flag for extreme duration trips

4. ✓ Handled outliers:
   - Age outliers (< 0 or > 120): Capped to reasonable range
   - Duration outliers: Flagged for investigation

5. ✓ Removed duplicates

6. ✓ Exported cleaned dataset to `data/processed/fordgobike_processed.csv`

### Dataset Statistics:
- **Original rows:** 183416 
- **Final rows:** 183412
- **Columns Before:** 16
- **Columns After:** 20
- **Missing values:** 0
- **Duplicates:** 0
